# Importing Libraries

In [78]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

In [79]:
import csv
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')  
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Importing Dataset

In [80]:
df = pd.read_csv('/content/spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# Data Preprocessing

In [82]:
df = df.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])


KeyError: ignored

In [83]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [84]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [85]:
df["v1"].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [86]:
df['v2'][3]

'U dun say so early hor... U c already then say...'

In [87]:
data[3] #3rd row

'u dun say earli hor u c alreadi say'

In [88]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [89]:
# Test and train spilit 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [90]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

# Create Model
# Add layers (LSTM ,Dense-(Hidden Layers),Ouput)

In [91]:
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

# Compile the model

In [92]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

# Fit the Model

In [93]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 12s 316ms/step - loss: 0.3428 - accuracy: 0.8672 - val_loss: 0.1778 - val_accuracy: 0.9198
Epoch 2/10
30/30 [==============================] - 8s 264ms/step - loss: 0.0900 - accuracy: 0.9789 - val_loss: 0.0914 - val_accuracy: 0.9726
Epoch 3/10
30/30 [==============================] - 8s 264ms/step - loss: 0.0429 - accuracy: 0.9892 - val_loss: 0.0467 - val_accuracy: 0.9873
Epoch 4/10
30/30 [==============================] - 8s 265ms/step - loss: 0.0294 - accuracy: 0.9921 - val_loss: 0.0520 - val_accuracy: 0.9852
Epoch 5/10
30/30 [==============================] - 8s 265ms/step - loss: 0.0222 - accuracy: 0.9945 - val_loss: 0.0794 - val_accuracy: 0.9778
Epoch 6/10
30/30 [==============================] - 8s 264ms/step - loss: 0.0208 - accuracy: 0.9937 - val_loss: 0.0556 - val_accuracy: 0.9842
Epoch 7/10
30/30 [==============================] - 8s 263ms/step - loss: 0.0144 - accuracy: 0.9958 - val_loss: 0.0709 - val_accuracy: 0.9821
Epoch

# Saving the model

In [94]:
model.save("spam.h5")

# Testing the Model

In [95]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [96]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 23ms/step - loss: 0.1212 - accuracy: 0.9809
Accuracy: 0.981


In [101]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred[1:25].round(3))

27/27 [==============================] - 2s 60ms/step
[[0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.176]
 [0.04 ]
 [0.   ]
 [1.   ]
 [1.   ]
 [0.   ]
 [1.   ]
 [0.005]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.   ]
 [1.   ]
 [0.   ]
 [0.   ]
 [0.   ]]


In [99]:
print(Y_test[1:25])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]]
